# Interactive Visualization of Minecraft Structure Spawning Probabilities

This notebook provides an interactive tool to explore the relationships between bounding boxes and spawnable areas for various structures in Minecraft. Using a combination of `matplotlib` and `ipywidgets`, the tool allows users to visualize how bounding boxes interact with the spawnable quadrants defined by the structures' region size and chunk range.

## Features

- **Bounding Box Visualization**: See how a defined bounding box interacts with the structure's spawnable areas and overlapping regions.
- **Dynamic Structure Selection**: Choose from a variety of structures like Desert Pyramids, Jungle Temples, and Villages, each with its unique region size and chunk range.
- **Interactive Controls**: Adjust bounding box coordinates (`X_min`, `Z_min`, `X_max`, `Z_max`) and immediately see the changes in the plot.
- **Spawn Probability Calculation**: Displays probabilities for spawning one or more structures within the bounding box, based on the intersecting quadrants.

## Instructions

1. **Select a Structure**: Use the dropdown menu to choose a structure. Each structure has different spawn mechanics, defined by its `regionSize` and `chunkRange`.
2. **Adjust Bounding Box Coordinates**: Modify the bounding box dimensions using the input fields for `X_min`, `Z_min`, `X_max`, and `Z_max`. Constraints ensure logical bounds (e.g., `X_min <= X_max`).
3. **View the Plot**:
    - The red rectangle represents the bounding box.
    - Blue areas show portions of quadrants overlapping the bounding box.
    - Grey rectangles outlinregion boundaries"spawnable range."
4. **Interpret the Results**: 
    - The numbers inside each quadrant indicate the valid spawnable cells.
    - Probabilities of no spawn (`P(S_BB = 0)`) and at least one spawn (`P(S_BB ≥ 1)`) are displayedand regional constraints.


In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import ipywidgets as widgets
from ipywidgets import fixed, VBox, HBox, Layout
from IPython.display import display

# Define the structures and their region sizes and chunk ranges
STRUCTURES = [
    {"name": "Desert Pyramid", "regionSize": 32, "chunkRange": 24},
    {"name": "Igloo", "regionSize": 32, "chunkRange": 24},
    {"name": "Jungle Temple", "regionSize": 32, "chunkRange": 24},
    {"name": "Swamp Hut", "regionSize": 32, "chunkRange": 24},
    {"name": "Outpost", "regionSize": 32, "chunkRange": 24},
    {"name": "Village", "regionSize": 34, "chunkRange": 26},
    {"name": "Ocean Ruin", "regionSize": 20, "chunkRange": 12},
    {"name": "Shipwreck", "regionSize": 24, "chunkRange": 20}
]


def get_quadrants_intersecting_bbox(struct, bbox):
    """Get the quadrants and regions intersecting the bounding box.
    
    Parameters:
        struct (dict): A dictionary containing the structure's name, region size, and chunk range.
        bbox (tuple): A tuple containing the bounding box coordinates (xmin, zmin, xmax, zmax).
    
    Returns:
        tuple: A tuple containing the list of regions and the list of quadrants intersecting the bounding box.
    """
    quadrants = []
    regions = []
    for x in range(bbox[0] - struct["regionSize"], bbox[2] + struct["regionSize"], 2 * struct["regionSize"]):
        for z in range(bbox[1] - struct["regionSize"], bbox[3] + struct["regionSize"], 2 * struct["regionSize"]):
            region_x_origin = (x + struct["regionSize"]) // (2 * struct["regionSize"]) * 2 * struct["regionSize"]
            region_z_origin = (z + struct["regionSize"]) // (2 * struct["regionSize"]) * 2 * struct["regionSize"]
            quads = [
                (region_x_origin, region_z_origin, region_x_origin + struct["chunkRange"] - 1, region_z_origin + struct["chunkRange"] - 1),
                (region_x_origin - struct["regionSize"], region_z_origin, region_x_origin - (struct["regionSize"] - struct["chunkRange"] + 1), region_z_origin + struct["chunkRange"] - 1),
                (region_x_origin - struct["regionSize"], region_z_origin - struct["regionSize"], region_x_origin - (struct["regionSize"] - struct["chunkRange"] + 1), region_z_origin - (struct["regionSize"] - struct["chunkRange"] + 1)),
                (region_x_origin, region_z_origin - struct["regionSize"], region_x_origin + struct["chunkRange"] - 1, region_z_origin - (struct["regionSize"] - struct["chunkRange"] + 1))
            ]
            
            rintersection_xmin = max(region_x_origin - struct["regionSize"], bbox[0])
            rintersection_zmin = max(region_z_origin - struct["regionSize"], bbox[1])
            rintersection_xmax = min(region_x_origin + struct["regionSize"], bbox[2])
            rintersection_zmax = min(region_z_origin + struct["regionSize"], bbox[3])
            
            if rintersection_xmin < rintersection_xmax and rintersection_zmin < rintersection_zmax:
                regions.append((region_x_origin - struct["regionSize"], region_z_origin - struct["regionSize"], region_x_origin + struct["regionSize"], region_z_origin + struct["regionSize"]))

            for quad_xmin, quad_zmin, quad_xmax, quad_zmax in quads:
                intersection_xmin = max(quad_xmin, bbox[0])
                intersection_zmin = max(quad_zmin, bbox[1])
                intersection_xmax = min(quad_xmax, bbox[2])
                intersection_zmax = min(quad_zmax, bbox[3])

                if intersection_xmin < intersection_xmax and intersection_zmin < intersection_zmax:
                    num_cells = (intersection_xmax + 1 - intersection_xmin) * (intersection_zmax + 1 - intersection_zmin)
                    quadrants.append((intersection_xmin, intersection_zmin, intersection_xmax, intersection_zmax, quad_xmin, quad_zmin, quad_xmax, quad_zmax, num_cells))
    return regions, quadrants


def plot_bounding_box_and_quadrants(struct, xmin, zmin, xmax, zmax):
    """Plot the bounding box and quadrants intersecting the bounding box.
    
    Parameters:
        struct (dict): A dictionary containing the structure's name, region size, and chunk range.
        xmin (int): The minimum x-coordinate of the bounding box.
        zmin (int): The minimum z-coordinate of the bounding box.
        xmax (int): The maximum x-coordinate of the bounding box.
        zmax (int): The maximum z-coordinate of the bounding box.
    
    Returns:
        None
    """
    bbox = xmin, zmin, xmax, zmax
    regions, quadrants = get_quadrants_intersecting_bbox(struct, bbox)
    fig, ax = plt.subplots()
    plt.subplots_adjust(top=0.85, right=0.75)

    # Plot the bounding box
    bbox_patches = patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1], linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(bbox_patches)

    total_valid_cells = 0
    for ixmin, izmin, ixmax, izmax, qxmin, qzmin, qxmax, qzmax, num_cells in quadrants:
        # Plot the intersection portion
        intersection = patches.Rectangle((ixmin, izmin), ixmax - ixmin, izmax - izmin, linewidth=1, edgecolor='blue', facecolor='blue', alpha=0.3, zorder=10)
        ax.add_patch(intersection)

        # Plot the portion outside the bounding box as hollow
        full_quadrant = patches.Rectangle((qxmin, qzmin), qxmax - qxmin, qzmax - qzmin, linewidth=1, edgecolor='blue', facecolor='none', zorder=10)
        ax.add_patch(full_quadrant)

        # Label the number of valid cells
        ax.text((ixmin + ixmax) / 2, (izmin + izmax) / 2, f"{num_cells}", color="white", ha="center", va="center", fontsize=8, fontweight='bold', zorder=10)
        total_valid_cells += num_cells
        region_size = struct["regionSize"]
        
        # Calculate extended plot limits to cover all structure ranges
        min_x = min([x[0] for x in regions])
        max_x = max([x[2] for x in regions])
        min_z = min([x[1] for x in regions])
        max_z = max([x[3] for x in regions])
        # Calculate and label the number of invalid cells
        total_cells = (qxmax - qxmin + 1) * (qzmax - qzmin + 1)
        num_invalid_cells = total_cells - num_cells
        hollow_label_x = (qxmin + qxmax) / 2
        hollow_label_z = (qzmin + qzmax) / 2

        # Calculate an optimal position based on the bounding box's location
        if qxmax > ixmax :  # Hollow area to the right of bounding box
            hollow_label_x = (qxmax + ixmax) / 2
        elif ixmax < ixmax:  # Hollow area to the left of bounding box
            hollow_label_x = (qxmin + ixmin) / 2
        if qzmax > izmax:  # Hollow area above the bounding box
            hollow_label_z = (qzmax + izmax) / 2
        else:  # Hollow area below the bounding box
            hollow_label_z = (qzmin + izmin) / 2
        ax.text(hollow_label_x, hollow_label_z, f"{num_invalid_cells}",
                color="black", ha="center", va="center", fontsize=8, fontweight='bold', zorder=10)

    # Plot the "r" range rectangle
    r_range = patches.Rectangle((-struct["regionSize"], -struct["regionSize"]), 2 * struct["regionSize"], 2 * struct["regionSize"], linewidth=2, edgecolor='grey', facecolor='none')
    ax.add_patch(r_range)

    # Set plot limits to show all structure ranges within extended bounds
    ax.set_xlim([min_x - 5, max_x + 5])
    ax.set_ylim([max_z + 5, min_z - 5])  # Invert z-axis for typical coordinates
    ax.set_aspect('equal', adjustable='box')

    # Plot additional "structure range" rectangles based on bounding box intersections
    for x in range(min_x, max_x, 2 * region_size):
        for z in range(min_z, max_z, 2 * region_size):
            # Each intersecting region center is (x, z) - Plot structure range rectangle
            range_rect = patches.Rectangle((x, z), 2 * region_size, 2 * region_size, linewidth=2, edgecolor='grey', facecolor='none')
            ax.add_patch(range_rect)
    # Set plot limits to show all structure ranges within extended bounds
    ax.set_xlim([min_x - 5, max_x + 5])
    ax.set_ylim([max_z + 5, min_z - 5])  # Invert z-axis for typical coordinates
            
    # Labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Z')
    plt.suptitle(f'{struct["name"]} Spawn Probabilities within Bounding Box')
    ax.set_title(f'{struct["name"]} range/chunksize={struct["regionSize"]}/{struct["chunkRange"]}; Bounding box=({bbox[0]},{bbox[1]}),({bbox[2]},{bbox[3]})', fontsize=8)

    # Set axis ticks every 2 units
    ax.set_xticks(range(int(min_x)// 5 * 5, (int(max_x)// 5) * 5 + 10, 5))
    ax.set_yticks(range(int(min_z)// 5 * 5, (int(max_z)// 5) * 5 + 10, 5))
    ax.tick_params(axis='both', which='major', labelsize=8)

    plt.grid(True)
    # Legend for different areas
    legend_elements = [
        patches.Patch(edgecolor='grey', facecolor='none', label="Structure Range", linewidth=2),
        patches.Patch(edgecolor='blue', facecolor='none', label="Structure Spawnable Area", linewidth=1),
        patches.Patch(edgecolor='red', facecolor='none', label="Bounding Box", linewidth=2),
        patches.Patch(edgecolor='none', facecolor='blue', alpha=0.3, label="Spawnable Area within Bounding Box")
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)

    # Calculate probabilities and add as text to the right
    mult_terms = f'\\left(\\frac{{' + f'}}{{{struct['chunkRange']}^2}}\\right)\\left(\\frac{{'.join([str((struct['chunkRange']*struct['chunkRange'])-n[8]) for n in quadrants]) + f'}}{{{struct['chunkRange']}^2}}\\right)'
    prob_no_struct = math.prod([(struct["chunkRange"]**2 - n[8]) / struct["chunkRange"]**2 for n in quadrants])
    prob_at_least_one_struct = 1 - prob_no_struct
    plt.text(1.05, 0.5, r'$\mathrm{P}(\text{S}_{\text{BB}} = 0) = ' + f'{mult_terms}$ = ' + f'{prob_no_struct:.3f}\n' +
             r'$\mathrm{P}(\text{S}_{\text{BB}} \geq 1)= 1 - \mathrm{P}(\text{S}_{\text{BB}} = 0) = $' + f'{prob_at_least_one_struct:.3f}', transform=ax.transAxes, va='center')
    plt.show()


def call_plotting(struct, xminlabel, xmin, zminlabel, zmin, xmaxlabel, xmax, zmaxlabel, zmax):
    """Call the plotting function with the selected structure and bounding box coordinates.
    
    Parameters:
        struct (dict): A dictionary containing the structure's name, region size, and chunk range.
        xminlabel (str): The label for the minimum x-coordinate of the bounding box.
        xmin (int): The minimum x-coordinate of the bounding box.
        zminlabel (str): The label for the minimum z-coordinate of the bounding box.
        zmin (int): The minimum z-coordinate of the bounding box.
        xmaxlabel (str): The label for the maximum x-coordinate of the bounding box.
        xmax (int): The maximum x-coordinate of the bounding box.
        zmaxlabel (str): The label for the maximum z-coordinate of the bounding box.
        zmax (int): The maximum z-coordinate of the bounding box.
    
    Returns:
        None
    """
    plot_bounding_box_and_quadrants(struct, xmin, zmin, xmax, zmax)


# Define functions to enforce constraints
def update_xmax(*args):
    if xmin_text.value > xmax_text.value:
        xmax_text.value = xmin_text.value


def update_xmin(*args):
    if xmax_text.value < xmin_text.value:
        xmin_text.value = xmax_text.value


def update_zmax(*args):
    if zmin_text.value > zmax_text.value:
        zmax_text.value = zmin_text.value


def update_zmin(*args):
    if zmax_text.value < zmin_text.value:
        zmin_text.value = zmax_text.value


bbox = (-30, -20, 10, 16)  # Default bounding box; modify as needed
# Dropdown widget to select structure
structure_dropdown = widgets.Dropdown(
    options=[(structure["name"], structure) for structure in STRUCTURES],
    value=STRUCTURES[0],  # Default value is the first structure
    description="Structure:"
)
# Create label for Bounding Box
bounding_box_label = widgets.Label(value="Bounding Box:")

# Create text inputs for bounding box coordinates
xmin_text = widgets.IntText(value=-30, description="", style={'description_width': '0'})
xmin_text.layout.width = '9rem'
xmax_text = widgets.IntText(value=10, description="", style={'description_width': '0'})
xmax_text.layout.width = '9rem'
zmin_text = widgets.IntText(value=-20, description="", style={'description_width': '0'})
zmin_text.layout.width = '9rem'
zmin_text.layout.margin = '0'
zmax_text = widgets.IntText(value=16, description="", style={'description_width': '0'})
zmax_text.layout.width = '9rem'
zmax_text.layout.margin = '0'
# Create an HTML widget for the label with subscript
xmin_label = widgets.HTML(value="X<sub>min</sub>", description="", style={'description_width': '0'})
xmin_label.layout.width = '3rem'
zmin_label = widgets.HTML(value="Z<sub>min</sub>", description="", style={'description_width': '0'})
zmin_label.layout.width = '3rem'
zmin_label.layout.margin = '0'
xmax_label = widgets.HTML(value="X<sub>max</sub>", description="", style={'description_width': '0'})
xmax_label.layout.width = '3rem'
zmax_label = widgets.HTML(value="Z<sub>max</sub>", description="", style={'description_width': '0'})
zmax_label.layout.width = '3rem'
zmax_label.layout.margin = '0'

# Set up observers for the text inputs
xmin_text.observe(update_xmax, 'value')
xmax_text.observe(update_xmin, 'value')
zmin_text.observe(update_zmax, 'value')
zmax_text.observe(update_zmin, 'value')

# Use ipywidgets.interactive to link the widget to the function
interactive_plot = widgets.interactive(call_plotting, struct=structure_dropdown, xminlabel=xmin_label, 
    xmin=xmin_text, zminlabel=zmin_label,
    zmin=zmin_text, xmaxlabel=xmax_label,
    xmax=xmax_text, zmaxlabel=zmax_label,
    zmax=zmax_text)

# Set up the layout of the widgets
interactive_plot.children[0].layout.width = '24rem'
interactive_plot.children[0].layout.margin = '0 0 0.5rem 0.5rem'
controls = VBox([HBox([interactive_plot.children[0]], layout = Layout(flex_flow='row wrap')),
                HBox([
                    HBox(interactive_plot.children[1:3], layout = Layout(flex_flow='row wrap')),
                    HBox(interactive_plot.children[3:5], layout = Layout(flex_flow='row wrap'))
                ]),
                HBox([
                    HBox(interactive_plot.children[5:7], layout = Layout(flex_flow='row wrap')),
                    HBox(interactive_plot.children[7:9], layout = Layout(flex_flow='row wrap'))
                ])])

# Display the interactive plot
output = interactive_plot.children[-1]
display(VBox([controls, output]))
